# Ejercicio prueba
Consta en hacer simples tareas de pandas pero ahora con la sintaxis de polars.
Los csv de emisiones-2016.csv, emisiones-2017.csv, emisiones-2018.csv y emisiones-2019.csv, contienen datos sobre las emisiones contaminates en la ciudad de Madrid en los años 2016, 2017, 2018 y 2019 respectivamente. Escribir un programa con los siguientes requisitos:

*Es importante que el código ea breve, ya que cada ejercicio puede hacerse en una sola instrucción* :OO

### Generar un DataFrame con los datos de los cuatro ficheros.
Hint: leer los datos y concatenarlos verticalmente --> **pl.concat**

In [ ]:
import polars as pl


e2016 = pl.read_csv('emisiones-2016.csv', separator= ';')
e2017 = pl.read_csv('emisiones-2017.csv', separator= ';')
e2018 = pl.read_csv('emisiones-2018.csv', separator= ';')
e2019 = pl.read_csv('emisiones-2019.csv', separator= ';')
emi = pl.concat([e2016,e2017, e2018,  e2019],how="vertical")
emi

### Filtrar las columnas del DataFrame para quedarse únicamente con las columnas de ESTACION, MAGNITUD, AÑO, MES y las correspondientes a todos los días D01, D02, ... 
Hint: **df.select(...)** 

In [ ]:
columnas = ['ESTACION', 'MAGNITUD', 'ANO', 'MES'] + [col for col in emi.columns if col.startswith('D')]
emi = emi.select(columnas)
emi

### Reestructurar el DataFrame para que los valores de los contaminantes de las columnas de los días aparezcan en una única columna.


In [ ]:
emi = emi.melt(id_vars=['ESTACION', 'MAGNITUD', 'ANO', 'MES'], variable_name='DIA', value_name='VALOR')
emi

### Crear una nueva columna llamada FECHA con las fechas a partir del año, mes y día
Hint: **pl.concat_str([
            pl.col(...),
            ...
        ])** y **.str.strptime(pl.Datetime,strict=False)** 

In [ ]:
emi = emi.select(
    [
        'ESTACION',
        'MAGNITUD',
        pl.concat_str([pl.col('ANO'),pl.col('MES'),pl.col('DIA').str.strip('D')], separator='-').str.strptime(pl.Datetime,'%Y-%m-%d', strict=False).alias('FECHA'),
        'VALOR'
    ])
emi

### Eliminar las filas con fechas no válidas

In [ ]:
emi = emi.filter(pl.col('FECHA') != None)
emi

### Ordenar el el dataframe por estación, magnitud y fecha
Hint: **.sort(..)**

In [ ]:
emi.sort(['ESTACION', 'MAGNITUD', 'FECHA'])

### Mostrar las estaciones y los contaminantes disponibles


In [ ]:
print('estaciones:', emi['ESTACION'].unique())
print('contaminantes:', emi['MAGNITUD'].unique())

## Tiempo de ejecución
Realiza una comparación de tiempo de ejecución utilizando pandas y polars. Lee todos los csv de https://github.com/pola-rs/polars/tree/main/examples/datasets, concatena verticalmente todos los csv y realiza un groupby a 'category' con cualquier función de agregación. De ahí compara el tiempo realizando 10000 iteraciones con la libreria `timeit` de python.

In [ ]:
import pandas as pd